# Car Price Prediction in the German Used Market

## 1. 📌 Introduction
- Objective: To analyze patterns in the German used car market and build a data-driven predictive model for estimating resale prices. The model will leverage key vehicle attributes—such as make, model, mileage (km driven), fuel type, and condition—to support pricing strategies, identify market trends, and inform decision-making for dealerships and online platforms.
- Dataset Source: Autoscout24.de or Kaggle  
- Features:
  - Brand: The brand or manufacturer of the car.
  - Model: The specific model of the car.
  - Color: The color of the car's exterior.
  - Registration Date: The date when the car was registered (Month/Year).
  - Year of Production: The year in which the car was manufactured.
  - Price in Euro: The price of the car in Euros.
  - Power: The power of the car in kilowatts (kW) and horsepower (ps).
  - Transmission Type: The type of transmission (e.g., automatic, manual).
  - Fuel Type: The type of fuel the car requires.
  - Fuel Consumption: Information about the car's fuel consumption in L/100km ang g/km.
  - Mileage: The total distance traveled by the car in km.
  - Offer Description: Additional description provided in the car offer.   
- Tools & Libraries Used
  - Python, Jupyter Notebbok
  - Libraries: Python Pandas, Numpy, SciPy, Scikit, Seaborn, Matplotlib
    


## 2. Data Cleaning

The following section deals with reading the dataset, cleaning the dataset of duplicated values and handling missing or null values, formatting the incorrect values like datetime, fuel_comsumption into a standard format

In [130]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy
from scipy import stats

- **Loading the Dataset**

In [132]:
df_main = pd.read_csv("cars_data.csv", sep=";")
df_main.head(3)

,Column1,brand,model,color,registration_date,year,price_in_euro,power_kw,power_ps,transmission_type,fuel_type,fuel_consumption_l_100km,fuel_consumption_g_km,mileage_in_km,offer_description
0,0,alfa-romeo,Alfa Romeo GTV,red,01/10/1995,1995.0,1300.0,148.0,201.0,Manual,Petrol,"10,9 l/100 km",260 g/km,1605000.0,2.0 V6 TB
1,1,alfa-romeo,Alfa Romeo 164,black,01/02/1995,1995.0,24900.0,191.0,260.0,Manual,Petrol,NaN,- (g/km),1900000.0,"Q4 Allrad, 3.2L GTA"
2,2,alfa-romeo,Alfa Romeo Spider,black,01/02/1995,1995.0,5900.0,110.0,150.0,Unknown,Petrol,NaN,- (g/km),1290000.0,ALFA ROME 916


In [133]:
df_main.shape

(251079, 15)

We will drop the column Unnamed:0 as it is of no significant use to us

In [135]:
df_main.drop(["Column1"], axis=1, inplace=True)

In [136]:
df_main.columns.tolist()

['brand',
 'model',
 'color',
 'registration_date',
 'year',
 'price_in_euro',
 'power_kw',
 'power_ps',
 'transmission_type',
 'fuel_type',
 'fuel_consumption_l_100km',
 'fuel_consumption_g_km',
 'mileage_in_km',
 'offer_description']

- **Finding and Dealing with the missing values**

In [138]:
df_main.isnull().any()

brand                       False
model                       False
color                        True
registration_date            True
year                         True
price_in_euro                True
power_kw                     True
power_ps                     True
transmission_type           False
fuel_type                   False
fuel_consumption_l_100km     True
fuel_consumption_g_km       False
mileage_in_km                True
offer_description            True
dtype: bool

In [139]:
df_main.isnull().sum()

brand                           0
model                           0
color                         166
registration_date             192
year                          195
price_in_euro                 199
power_kw                      293
power_ps                      210
transmission_type               0
fuel_type                       0
fuel_consumption_l_100km    26873
fuel_consumption_g_km           0
mileage_in_km                 152
offer_description               1
dtype: int64

Based on the columns and according to the needs of our analyses, we will drop the rows of missing values from registration_date and offer_description since such less null values will not affect our analysis strongly. We will also drop the missing values rows from fuel_consumption_l_100km since more than 90% of the values are missing. We will use the alternate substitute which is fuel_comsumption_g_km. 

In [141]:
df_main.dropna(subset=["registration_date", "fuel_consumption_l_100km", "offer_description"], inplace=True)

In [142]:
df_main.isnull().sum()

brand                         0
model                         0
color                       150
registration_date             0
year                          7
price_in_euro                 7
power_kw                     76
power_ps                     83
transmission_type             0
fuel_type                     0
fuel_consumption_l_100km      0
fuel_consumption_g_km         0
mileage_in_km                55
offer_description             0
dtype: int64

df_main["color"].fillna(df_main["color"].mode()[0], inplace=True)
df_main["power_kw"].fillna(df_main["power_kw"].mode()[0], inplace=True)
df_main["power_ps"].fillna(df_main["power_ps"].mode()[0], inplace=True)
df_main["mileage_in_km"].fillna(df_main["mileage_in_km"].mode()[0], inplace=True)

In [144]:
df_main.head(3)

,brand,model,color,registration_date,year,price_in_euro,power_kw,power_ps,transmission_type,fuel_type,fuel_consumption_l_100km,fuel_consumption_g_km,mileage_in_km,offer_description
0,alfa-romeo,Alfa Romeo GTV,red,01/10/1995,1995.0,1300.0,148.0,201.0,Manual,Petrol,"10,9 l/100 km",260 g/km,1605000.0,2.0 V6 TB
3,alfa-romeo,Alfa Romeo Spider,black,01/07/1995,1995.0,4900.0,110.0,150.0,Manual,Petrol,"9,5 l/100 km",225 g/km,1895000.0,2.0 16V Twin Spark L
4,alfa-romeo,Alfa Romeo 164,red,01/11/1996,1996.0,17950.0,132.0,179.0,Manual,Petrol,"7,2 l/100 km",- (g/km),961270.0,"3.0i Super V6, absoluter Topzustand !"


- **Converting the incorrect values into correct ones with calculations**

In [146]:
# To convert the fuel consumption g/km to a numerical column
df_main["fuel_consumption_g_km"] = df_main["fuel_consumption_g_km"].str.extract(r'(\d+\.?\d*)').astype(float) 

In [147]:
df_main.head()

,brand,model,color,registration_date,year,price_in_euro,power_kw,power_ps,transmission_type,fuel_type,fuel_consumption_l_100km,fuel_consumption_g_km,mileage_in_km,offer_description
0,alfa-romeo,Alfa Romeo GTV,red,01/10/1995,1995.0,1300.0,148.0,201.0,Manual,Petrol,"10,9 l/100 km",260.0,1605000.0,2.0 V6 TB
3,alfa-romeo,Alfa Romeo Spider,black,01/07/1995,1995.0,4900.0,110.0,150.0,Manual,Petrol,"9,5 l/100 km",225.0,1895000.0,2.0 16V Twin Spark L
4,alfa-romeo,Alfa Romeo 164,red,01/11/1996,1996.0,17950.0,132.0,179.0,Manual,Petrol,"7,2 l/100 km",NaN,961270.0,"3.0i Super V6, absoluter Topzustand !"
5,alfa-romeo,Alfa Romeo Spider,red,01/04/1996,1996.0,7900.0,110.0,150.0,Manual,Petrol,"9,5 l/100 km",225.0,473070.0,2.0 16V Twin Spark
6,alfa-romeo,Alfa Romeo 145,red,01/12/1996,1996.0,3500.0,110.0,150.0,Manual,Petrol,"8,8 l/100 km",210.0,2300000.0,Quadrifoglio


In [148]:
df_main["fuel_consumption_l_100km"] = df_main["fuel_consumption_l_100km"].str.extract(r'(\d+\.?\d*)').astype(float) 

In [149]:
df_main.head()

,brand,model,color,registration_date,year,price_in_euro,power_kw,power_ps,transmission_type,fuel_type,fuel_consumption_l_100km,fuel_consumption_g_km,mileage_in_km,offer_description
0,alfa-romeo,Alfa Romeo GTV,red,01/10/1995,1995.0,1300.0,148.0,201.0,Manual,Petrol,10.0,260.0,1605000.0,2.0 V6 TB
3,alfa-romeo,Alfa Romeo Spider,black,01/07/1995,1995.0,4900.0,110.0,150.0,Manual,Petrol,9.0,225.0,1895000.0,2.0 16V Twin Spark L
4,alfa-romeo,Alfa Romeo 164,red,01/11/1996,1996.0,17950.0,132.0,179.0,Manual,Petrol,7.0,NaN,961270.0,"3.0i Super V6, absoluter Topzustand !"
5,alfa-romeo,Alfa Romeo Spider,red,01/04/1996,1996.0,7900.0,110.0,150.0,Manual,Petrol,9.0,225.0,473070.0,2.0 16V Twin Spark
6,alfa-romeo,Alfa Romeo 145,red,01/12/1996,1996.0,3500.0,110.0,150.0,Manual,Petrol,8.0,210.0,2300000.0,Quadrifoglio


Since there are missing values in the fuel_consumpiton_g_km columns. We can calculate those values from fuel_consumption_l_100km column and replace those values by the following formula for calculation:


$$
\text{Fuel Consumption (g/km)} = \frac{\text{L/100km} \times \text{Fuel Density (kg/L)} \times 1000}{100}
$$

Where:
- **L/100km** is the fuel usage
- **Fuel Density** ≈ 0.745 for petrol, 0.832 for diesel
- `1000` converts kg to grams

In [151]:
petrol_density = 0.745  # kg/L
diesel_density = 0.832  # kg/L

def estimate_g_km(row):
    if pd.isna(row["fuel_consumption_g_km"]) and not pd.isna(row["fuel_consumption_l_100km"]):
        if row["fuel_type"].lower() == "petrol":
            return (row["fuel_consumption_l_100km"] * petrol_density * 1000) / 100
        elif row["fuel_type"].lower() == "diesel":
            return (row["fuel_consumption_l_100km"] * diesel_density * 1000) / 100
    return row["fuel_consumption_g_km"]

df_main["fuel_consumption_g_km"] = df_main.apply(estimate_g_km, axis=1)

We can also find the main model names of the cars from the model column for an overviewing

In [153]:
df_main["brand"].nunique()

47

In [154]:
df_main["brand"].unique().tolist()

['alfa-romeo',
 'aston-martin',
 'audi',
 'bentley',
 'bmw',
 'cadillac',
 'chevrolet',
 'chrysler',
 'citroen',
 'dacia',
 'daewoo',
 'daihatsu',
 'dodge',
 'ferrari',
 'fiat',
 'ford',
 'honda',
 'hyundai',
 'infiniti',
 'isuzu',
 'jaguar',
 'jeep',
 'kia',
 'lada',
 'lamborghini',
 'lancia',
 'land-rover',
 'maserati',
 'mazda',
 'mercedes-benz',
 'mini',
 'mitsubishi',
 'nissan',
 'opel',
 'peugeot',
 'porsche',
 'proton',
 'renault',
 'rover',
 'saab',
 'seat',
 'skoda',
 'smart',
 'ssangyong',
 'toyota',
 'volkswagen',
 'volvo']

Here we can see that there are main 47 brands of cars in the dataset each of them having their respective models names. They all are already arranged in an alphabetical order

We will just check the duplicated values to avoid any errors and repatation

In [157]:
df_main.duplicated()

0         False
3         False
4         False
5         False
6         False
          ...  
251071    False
251072    False
251073    False
251075    False
251076    False
Length: 224014, dtype: bool

In [158]:
#Converting the Date format
df_main["registration_date"] = df_main["registration_date"].str.replace("/", ".", regex=False)
df_main.head(3)

,brand,model,color,registration_date,year,price_in_euro,power_kw,power_ps,transmission_type,fuel_type,fuel_consumption_l_100km,fuel_consumption_g_km,mileage_in_km,offer_description
0,alfa-romeo,Alfa Romeo GTV,red,01.10.1995,1995.0,1300.0,148.0,201.0,Manual,Petrol,10.0,260.00,1605000.0,2.0 V6 TB
3,alfa-romeo,Alfa Romeo Spider,black,01.07.1995,1995.0,4900.0,110.0,150.0,Manual,Petrol,9.0,225.00,1895000.0,2.0 16V Twin Spark L
4,alfa-romeo,Alfa Romeo 164,red,01.11.1996,1996.0,17950.0,132.0,179.0,Manual,Petrol,7.0,52.15,961270.0,"3.0i Super V6, absoluter Topzustand !"


In [196]:
df_main = df_main[df_main["mileage_in_km"] != 0]

In [198]:
(df_main["mileage_in_km"] == 0).sum()

0

- **Saving the cleaned Dataset**

In [194]:
df_main.to_csv("cars_dataset_cleaned.csv", index=False)
print("The cleaned dataset has been saved")

The cleaned dataset has been saved
